In [ ]:
# !pip install awswrangler
#restart kernel before using this

In [2]:
import boto3
import botocore
import logging
from botocore.exceptions import ClientError
import pandas as pd
from IPython.display import display, Markdown
import awswrangler as wr

s3_client = boto3.client('s3')
s3_resource = boto3.resource('s3')

logging.basicConfig(level=logging.INFO)

In [3]:
buckets = s3_client.list_buckets()
print('Existing buckets:')
for bucket in buckets['Buckets']:
    print(f'  {bucket["Name"]}')

Existing buckets:
  aws-glue-scripts-476680029350-eu-west-1
  aws-glue-temporary-476680029350-eu-west-1
  rapid-dw


In [4]:
def list_buckets(match=''):
    response = s3_client.list_buckets()
    if match:
        print(f'Existing buckets containing "{match}" string:')
    else:
        print('All existing buckets:')
    for bucket in response['Buckets']:
        if match:
            if match in bucket["Name"]:
                print(f'  {bucket["Name"]}')
        else:
                print(f'  {bucket["Name"]}')

list_buckets(match='rapid')

Existing buckets containing "rapid" string:
  rapid-dw


In [5]:
bucket_resource = s3_resource.Bucket('rapid-dw')
for file in bucket_resource.objects.all():
    print(f'{file.key}')

2020-10-08_Gesamt/POCTIKWH.EBKVSD
2020-10-08_Gesamt/POCTIKWH.MAKT.G
2020-10-08_Gesamt/POCTIKWH.MARA.G
2020-10-08_Gesamt/POCTIKWH.VSFR.G
2020-10-08_Gesamt/POCTIKWH.VSKB.G
2020-10-08_Gesamt/POCTIKWH.VSKL.G
2020-10-08_Gesamt/POCTIKWH.VSKN.G
2020-10-08_Gesamt/POCTIKWH.VSKT.A
2020-10-08_Gesamt/POCTIKWH.VSKT.G
2020-10-08_Gesamt/POCTIKWH.VSSP.G
2020-10-08_Gesamt/POCTIKWH.VSTE.G
2020-10-08_Gesamt/POCTIKWH.VSVA.G
2020-10-08_Gesamt/POCTIKWH.VSVT.A
2020-10-08_Gesamt/POCTIKWH.VSVT.G


In [6]:
def download_file(file_name, bucket, object_name=None):
    """Download a file from an S3 bucket

    :param file_name: File to upload
    :param bucket: Bucket to upload to
    :param object_name: S3 object name. If not specified then file_name is used
    :return: True if file was uploaded, else False
    """

    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = file_name

    # Download the file
    s3_client = boto3.client('s3')
    
    logging.debug(f'bucket={bucket}\nobject_name={object_name}\nfile_name={file_name}')
    
    try:
        response = s3_client.download_file(bucket, object_name, file_name)
    except ClientError as e:
        logging.error(e)
        return False
    return True

bucket = 'rapid-dw'
folder_name = '2020-10-08_Gesamt'
file_name = 'POCTIKWH.VSKT.G'
object_name = folder_name + '/' + file_name
# response = download_file(file_name, bucket, object_name)

# print(f'reponse={response}')

In [7]:
bucket='rapid-dw'
file='2020-10-08_Gesamt/POCTIKWH.VSVA.G'
data_source = {
        'Bucket': bucket,
        'Key': file
    }
# Generate the URL to get Key from Bucket
url = s3_client.generate_presigned_url(
    ClientMethod = 'get_object',
    Params = data_source
)

#data = pd.read_csv(url, nrows=10)
# widths = [4,1,18,32,12,32,8,8,1,18,13,3,1,5,4,2,1,2,1,3,3,6,3,4,1,1,6,1,3,1,3,3,13,1,4,4,4,4,4,4,4,4,4,4,2,2,2,1,6,6,1,1,12,30,8,8,8,8,8,8,1,1,3,3,60,1,1,1,1,32,8,3,8,2,8,2,1,1]
# header = ['satzart','uid','classf','pvguid','aennr','pnguid','pvname','pvtype','loekz','matnr','quant','qunit','fe_ae_st','abw_schl','aggr','auf_schl','dummy15','dmu_rel','mee','mod','paket_kz','dummy1','dummy2','kostgr','gew_stk','tma_klag','arb_pkt','kd_kz','mod_z','konstgr','se_team','pr_stkz','taed_bem','dummy4','dummy5','dummy6','dummy7','dummy8','dummy9','dummy10','dummy11','dummy12','dummy13','wwfall','wahlw_nr','vwwwfall','vwwwnr','leerteil','dummy18','dummy19','dummy20','dummy_v','dummy22','dummy23','p_target_date','g_target_date','w_target_date','b_target_date','r_target_date','d_target_date','bereich','u_abteil','anlart','se_teamp','pvtext','mj_ein','mj_aus','bkflg','bk_status','dummy3','seqcnt','kar_aenr','vsi_b_date','vsi_b_chg','vsi_k_date','vsi_k_chg','prio_mark','herkunft']
#data = pd.read_fwf(url, widths=widths, header=None, nrows=10)
# data = pd.read_fwf(url, widths=widths, header=None)
# data.info()

In [8]:
header_vskl=['satzart','uid','classf','class','clintf','clint','klagr','txtbz']
header_vskb=['satzart','uid','classf','pnguid_s','pnguid_v','rank','herkunft']
header_vsva=['satzart','uid','classf','pvguid','aennr','pnguid','pvname','pvtype','loekz','matnr','quant','qunit','fe_ae_st','abw_schl','aggr','auf_schl','dummy15','dmu_rel','mee','mod','paket_kz','dummy1','dummy2','kostgr','gew_stk','tma_klag','arb_pkt','kd_kz','mod_z','konstgr','se_team','pr_stkz','taed_bem','dummy4','dummy5','dummy6','dummy7','dummy8','dummy9','dummy10','dummy11','dummy12','dummy13','wwfall','wahlw_nr','vwwwfall','vwwwnr','leerteil','dummy18','dummy19','dummy20','dummy_v','dummy22','dummy23','p_target_date','g_target_date','w_target_date','b_target_date','r_target_date','d_target_date','bereich','u_abteil','anlart','se_teamp','pvtext','mj_ein','mj_aus','bkflg','bk_status','dummy3','seqcnt','kar_aenr','vsi_b_date','vsi_b_chg','vsi_k_date','vsi_k_chg','prio_mark','herkunft']
header_vskt=['satzart','uid','pnguid','spras','pntext','herkunft']
header_vskn=['satzart','uid','pnguid','classf','pname','pntype','class','geb_aufl','kostgr_k','konstr_k','st_name','se_teamk','pr_stkzk','fogs_kz','bereichk','uabteilk','pfr_kz','pfr_user','pfr_komm','gfr_kz','gfr_user','gfr_komm','wfr_kz','wfr_user','wfr_komm','bfr_kz','bfr_user','bfr_komm','rfr_kz','rfr_user','rfr_komm','dfr_kz','dfr_user','dfr_komm','kbblock','mkguid','pntext','diag_adr','herkunft','pnguid_q','kps_clint','kps_class']
header_vssp=['satzart','uid','classf','pvguid','aennr','splitcnt','pnguidnr','einsatzint','einsatzext','entfallint','entfallext','tegue','avon_status','vws_tmp','vws','col_pnguid','col_bst','col_family','takt','umfang','urguid','teileart','verbr1','verbr2','verbr3','verbr4','verbr5','verbr6','fert1','fert2','fert3','herkunft','bea_st','taed_bem','taktvo','taktna','feaest_t']
header_vste=['satzart','uid','classf','pvguid','aennr','splitcnt','datum_pvs_low','datum_ns_low','datum_sop_low','datub_pvs_fr','datub_ns_fr','datub_sop_fr','datub_pvs_sp','datub_ns_sp','datub_sop_sp','herkunft','errflg']
header_vsfr=['satzart','uid','pvguid','classf','aennr','splitcnt','frart','value','vgnum','infotr','franr','frnnr','fdate','ftime','aeind','dmupru','dummy','ftarg','frsta','frzte','marke','afreis','frben','herkunft']
widths_vskl=[4,1,18,18,10,10,10,60]
widths_vskb=[4,1,18,32,32,4,1]
widths_vsva=[4,1,18,32,12,32,8,8,1,18,13,3,1,5,4,2,1,2,1,3,3,6,3,4,1,1,6,1,3,1,3,3,13,1,4,4,4,4,4,4,4,4,4,4,2,2,2,1,6,6,1,1,12,30,8,8,8,8,8,8,1,1,3,3,60,1,1,1,1,32,8,3,8,2,8,2,1,1]
widths_vskt=[4,1,32,2,60,1]
widths_vskn=[4,1,32,18,40,8,18,1,4,1,36,3,3,2,1,1,1,12,80,1,12,80,1,12,80,1,12,80,1,12,80,1,12,80,9,32,60,4,1,32,10,18]
widths_vssp=[4,1,18,32,12,8,6,12,30,12,30,80,4,9,9,32,1,3,6,3,32,1,4,4,4,4,4,4,4,4,4,1,1,13,6,6,1]
widths_vste=[4,1,18,32,12,8,8,8,8,8,8,8,8,8,8,1,1]
widths_vsfr=[4,1,32,18,12,8,1,3,18,18,12,2,8,6,4,3,40,8,10,8,20,12,80,1]

In [9]:
def read_file(bucket, file, widths=None, header=None):
    data_source = {
            'Bucket': bucket,
            'Key': file
        }
    # Generate the URL to get Key from Bucket
    url = s3_client.generate_presigned_url(
        ClientMethod = 'get_object',
        Params = data_source
    )

    compression="infer"
    encoding="utf-8"
    #dataframe = pd.read_fwf(url, widths=widths, header=header, nrows=10)
    dataframe = pd.read_fwf(url, widths=widths, header=None, names=header, compression=compression, encoding=encoding)
    return dataframe
    
result = read_file('rapid-dw','2020-10-08_Gesamt/POCTIKWH.VSTE.G', widths=widths_vste, header=header_vste)
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75020 entries, 0 to 75019
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   satzart        75020 non-null  object 
 1   uid            0 non-null      float64
 2   classf         75020 non-null  object 
 3   pvguid         75020 non-null  object 
 4   aennr          75020 non-null  int64  
 5   splitcnt       75020 non-null  int64  
 6   datum_pvs_low  75020 non-null  int64  
 7   datum_ns_low   75020 non-null  int64  
 8   datum_sop_low  75020 non-null  int64  
 9   datub_pvs_fr   75020 non-null  int64  
 10  datub_ns_fr    75020 non-null  int64  
 11  datub_sop_fr   75020 non-null  int64  
 12  datub_pvs_sp   75020 non-null  int64  
 13  datub_ns_sp    75020 non-null  int64  
 14  datub_sop_sp   75020 non-null  int64  
 15  herkunft       75020 non-null  object 
 16  errflg         2394 non-null   object 
dtypes: float64(1), int64(11), object(5)
memory usage: 

In [ ]:
result.head(1)

In [ ]:
# file_name="test.parquet"
# result.to_parquet(file_name, engine="pyarrow")

In [11]:
path = "working/inbound/vste.parquet"
wr.s3.to_parquet(
    df=result,
    path="s3://" + bucket + "/" + path
)



{'paths': ['s3://rapid-dw/working/inbound/vste.parquet'],
 'partitions_values': {}}

In [ ]:
# out = pd.read_parquet(file_name, engine="pyarrow")
# out.info()